In [1]:
import pandas as pd
import numpy as np
from time import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
CLEANED_FILE = 'cleaned_dataset.csv' 
df = pd.read_csv(CLEANED_FILE)
print("Loaded dataframe shape:", df.shape)
display(df.head())

Loaded dataframe shape: (29956, 20)


,respondent_id,gender,zone,occupation,income_levels,consume_frequency(weekly),current_brand,preferable_consumption_size,awareness_of_other_brands,reasons_for_choosing_brands,flavor_preference,purchase_channel,packaging_preference,health_concerns,typical_consumption_situations,price_range,age_group,cf_ab_score,zas_score,bsi
0,R00001,M,3,Working Professional,1,2,Newcomer,Medium (500 ml),1,Price,Traditional,Online,Simple,Medium (Moderately health-conscious),"Active (eg. Sports, gym)",100-150,26-35,0.67,3,1
1,R00002,F,4,Working Professional,5,3,Established,Medium (500 ml),2,Quality,Exotic,Retail Store,Premium,Medium (Moderately health-conscious),Social (eg. Parties),200-250,46-55,0.60,20,0
2,R00003,F,1,Working Professional,5,2,Newcomer,Medium (500 ml),2,Availability,Traditional,Retail Store,Premium,Medium (Moderately health-conscious),"Active (eg. Sports, gym)",200-250,36-45,0.50,5,0
3,R00004,F,3,Working Professional,3,3,Newcomer,Medium (500 ml),1,Brand Reputation,Exotic,Online,Eco-Friendly,Low (Not very concerned),"Active (eg. Sports, gym)",150-200,26-35,0.75,9,0
4,R00005,M,4,Student,0,2,Established,Medium (500 ml),1,Availability,Traditional,Online,Premium,Medium (Moderately health-conscious),"Active (eg. Sports, gym)",50-100,18-25,0.67,0,0


In [3]:
TARGET = "price_range"
if TARGET not in df.columns:
    raise ValueError(f"Target column '{TARGET}' not found in dataframe. Please ensure '{CLEANED_FILE}' contains '{TARGET}'.")

In [4]:
print("Columns:", df.columns.tolist())
print("\nNull counts:")
print(df.isnull().sum())
print("\nTarget value counts:")
print(df[TARGET].value_counts(dropna=False))

Columns: ['respondent_id', 'gender', 'zone', 'occupation', 'income_levels', 'consume_frequency(weekly)', 'current_brand', 'preferable_consumption_size', 'awareness_of_other_brands', 'reasons_for_choosing_brands', 'flavor_preference', 'purchase_channel', 'packaging_preference', 'health_concerns', 'typical_consumption_situations', 'price_range', 'age_group', 'cf_ab_score', 'zas_score', 'bsi']

Null counts:
respondent_id                     0
gender                            0
zone                              0
occupation                        0
income_levels                     0
consume_frequency(weekly)         0
current_brand                     0
preferable_consumption_size       0
awareness_of_other_brands         0
reasons_for_choosing_brands       0
flavor_preference                 0
purchase_channel                  0
packaging_preference              0
health_concerns                   0
typical_consumption_situations    0
price_range                       0
age_group       

In [5]:
if "respondent_id" in df.columns:
    df = df.drop(columns=["respondent_id"])

X = df.drop(columns=[TARGET]).copy()
y = df[TARGET].copy()

print("Feature columns count:", X.shape[1])


Feature columns count: 18


In [6]:
stratify_arg = y if (y.nunique() > 1 and y.dtype != "float") else None
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )
except Exception:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=None
    )

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

Train shape: (22467, 18) Test shape: (7489, 18)


In [7]:
label_encode_cols_requested = [
    "age_group",
    "income_levels",
    "health_concerns",
    "consume_frequency(weekly)",
    "preferable_consumption_size"
]

In [8]:
label_encode_cols = [c for c in label_encode_cols_requested if c in X_train.columns]
cat_cols_all = list(X_train.select_dtypes(include=["object", "category"]).columns)
onehot_cols = [c for c in cat_cols_all if c not in label_encode_cols]

print("Label-encode columns (present):", label_encode_cols)
print("One-hot columns (detected):", onehot_cols)


Label-encode columns (present): ['age_group', 'income_levels', 'health_concerns', 'consume_frequency(weekly)', 'preferable_consumption_size']
One-hot columns (detected): ['gender', 'occupation', 'current_brand', 'reasons_for_choosing_brands', 'flavor_preference', 'purchase_channel', 'packaging_preference', 'typical_consumption_situations']


In [9]:
le_target = LabelEncoder()
y_train_enc = le_target.fit_transform(y_train.astype(str))
y_test_enc = le_target.transform(y_test.astype(str))
target_classes = le_target.classes_
print("Target classes:", list(target_classes))

# Feature label encoders 
label_encoders = {}
for col in label_encode_cols:
    X_train[col] = X_train[col].fillna("___MISSING___").astype(str)
    X_test[col] = X_test[col].fillna("___MISSING___").astype(str)
    le = LabelEncoder()
    le.fit(X_train[col])
    X_train[col] = le.transform(X_train[col])
    test_vals = set(X_test[col].unique())
    unseen = [v for v in test_vals if v not in set(le.classes_)]
    if unseen:
        union = list(le.classes_) + ["___UNSEEN___"]
        le2 = LabelEncoder()
        le2.fit(union)
        X_train[col] = le2.transform(X_train[col].astype(str))
        X_test[col] = X_test[col].astype(str).map(lambda v: v if v in union else "___UNSEEN___")
        X_test[col] = le2.transform(X_test[col])
        label_encoders[col] = le2
    else:
        X_test[col] = le.transform(X_test[col])
        label_encoders[col] = le
print("Feature label encoding complete.")


Target classes: ['100-150', '150-200', '200-250', '50-100']
Feature label encoding complete.


In [10]:
if len(onehot_cols) > 0:
    for c in onehot_cols:
        X_train[c] = X_train[c].fillna("___MISSING___").astype(str)
        X_test[c] = X_test[c].fillna("___MISSING___").astype(str)

    X_train_ohe = pd.get_dummies(X_train, columns=onehot_cols, drop_first=False)
    X_test_ohe = pd.get_dummies(X_test, columns=onehot_cols, drop_first=False)

    X_train_ohe, X_test_ohe = X_train_ohe.align(X_test_ohe, join="left", axis=1, fill_value=0)

    X_train_ohe = X_train_ohe.fillna(0)
    X_test_ohe = X_test_ohe.fillna(0)
else:
    X_train_ohe = X_train.copy()
    X_test_ohe = X_test.copy()

print("After encoding shapes - Train:", X_train_ohe.shape, "Test:", X_test_ohe.shape)


After encoding shapes - Train: (22467, 32) Test: (7489, 32)


In [11]:
numeric_cols = X_train_ohe.select_dtypes(include=[np.number]).columns.tolist()
if len(numeric_cols) == 0:
    raise ValueError("No numeric columns found after encoding. Please check the data and encoding steps.")

imp = SimpleImputer(strategy="median")
scaler = StandardScaler()

X_train_num = imp.fit_transform(X_train_ohe[numeric_cols])
X_train_num = scaler.fit_transform(X_train_num)

X_test_num = imp.transform(X_test_ohe[numeric_cols])
X_test_num = scaler.transform(X_test_num)

X_train_final = pd.DataFrame(X_train_num, columns=numeric_cols, index=X_train_ohe.index)
X_test_final = pd.DataFrame(X_test_num, columns=numeric_cols, index=X_test_ohe.index)

print("Numeric imputation & scaling complete. Final feature count:", X_train_final.shape[1])


Numeric imputation & scaling complete. Final feature count: 10


In [12]:
X_train_arr = X_train_final.values
X_test_arr = X_test_final.values
y_train_arr = y_train_enc
y_test_arr = y_test_enc


In [13]:
models = {
    "Gaussian Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Support Vector Machine": SVC(random_state=42),         
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}


In [14]:
results = {}
def train_and_evaluate(name, model, Xtr, Xte, ytr, yte, target_names):
    t0 = time()
    model.fit(Xtr, ytr)
    tt = time() - t0
    preds = model.predict(Xte)
    acc = accuracy_score(yte, preds)
    report = classification_report(yte, preds, target_names=target_names, zero_division=0)
    print("\n" + "="*80)
    print(f"Model: {name}")
    print(f"Training time (s): {tt:.3f}")
    print(f"Accuracy: {acc:.4f}")
    print("Classification report|:")
    print(report)
    results[name] = {"model": model, "accuracy": acc, "report": report}


In [15]:
for name, model in models.items():
    try:
        train_and_evaluate(name, model, X_train_arr, X_test_arr, y_train_arr, y_test_arr, target_classes)
    except Exception as e:
        print("\n" + "-"*60)
        print(f"ERROR training/evaluating {name}: {e}")
        print("-"*60)



Model: Gaussian Naive Bayes
Training time (s): 0.014
Accuracy: 0.6321
Classification report|:
              precision    recall  f1-score   support

     100-150       0.53      0.35      0.42      1948
     150-200       0.59      0.58      0.58      2199
     200-250       0.84      0.81      0.82      2428
      50-100       0.48      0.89      0.63       914

    accuracy                           0.63      7489
   macro avg       0.61      0.66      0.61      7489
weighted avg       0.64      0.63      0.62      7489


Model: Logistic Regression
Training time (s): 0.409
Accuracy: 0.7642
Classification report|:
              precision    recall  f1-score   support

     100-150       0.71      0.72      0.71      1948
     150-200       0.70      0.70      0.70      2199
     200-250       0.87      0.87      0.87      2428
      50-100       0.78      0.73      0.75       914

    accuracy                           0.76      7489
   macro avg       0.76      0.76      0.76      7

C:\Users\KHUSH\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:199: UserWarning: [16:19:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Model: XGBoost
Training time (s): 4.233
Accuracy: 0.8495
Classification report|:
              precision    recall  f1-score   support

     100-150       0.82      0.81      0.82      1948
     150-200       0.80      0.81      0.81      2199
     200-250       0.91      0.91      0.91      2428
      50-100       0.85      0.86      0.86       914

    accuracy                           0.85      7489
   macro avg       0.85      0.85      0.85      7489
weighted avg       0.85      0.85      0.85      7489

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 60
[LightGBM] [Info] Number of data points in the train set: 22467, number of used features: 10
[LightGBM] [Info] Start training from score -1.346461
[LightGBM] [Info] Start training from score -1.225281
[LightGBM] [Info] Start 

C:\Users\KHUSH\AppData\Roaming\Python\Python313\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\KHUSH\AppData\Roaming\Python\Python313\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Program Files\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python313\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,


Model: LightGBM
Training time (s): 7.475
Accuracy: 0.8502
Classification report|:
              precision    recall  f1-score   support

     100-150       0.82      0.81      0.82      1948
     150-200       0.80      0.81      0.81      2199
     200-250       0.92      0.91      0.91      2428
      50-100       0.84      0.87      0.86       914

    accuracy                           0.85      7489
   macro avg       0.85      0.85      0.85      7489
weighted avg       0.85      0.85      0.85      7489



C:\Users\KHUSH\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [16]:
if results:
    summary = sorted([(name, info["accuracy"]) for name, info in results.items()], key=lambda x: x[1], reverse=True)
    print("\nModel performance summary (sorted by accuracy):")
    for name, acc in summary:
        print(f"{name:25s} -> Accuracy: {acc:.4f}")
else:
    print("No successful model runs to summarize.")



Model performance summary (sorted by accuracy):
LightGBM                  -> Accuracy: 0.8502
XGBoost                   -> Accuracy: 0.8495
Support Vector Machine    -> Accuracy: 0.8355
Random Forest             -> Accuracy: 0.8293
Logistic Regression       -> Accuracy: 0.7642
Gaussian Naive Bayes      -> Accuracy: 0.6321


## ML Flow

In [17]:
import mlflow
import mlflow.sklearn

In [18]:
# import mlflow, os

# USERNAME = " "
# REPO = " "
# TOKEN = " "   

# os.environ["MLFLOW_TRACKING_USERNAME"] = USERNAME
# os.environ["MLFLOW_TRACKING_PASSWORD"] = TOKEN

# EXPERIMENT_NAME = "Beverage Price Prediction"
# mlflow.set_experiment(EXPERIMENT_NAME)

# print("Tracking URI :", mlflow.get_tracking_uri())

In [19]:
EXPERIMENT_NAME = "Beverage Price Prediction"

mlflow.set_experiment(EXPERIMENT_NAME)

mlflow.set_tracking_uri("http://127.0.0.1:5000/")


C:\Users\KHUSH\AppData\Roaming\Python\Python313\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)


In [20]:
def log_to_mlflow(model_name, model_obj, accuracy, report, X_train, y_train):
    """
    Logs model, parameters, metrics, and classification report to MLflow (DagsHub compatible).
    """
    with mlflow.start_run(run_name=model_name):
        params = getattr(model_obj, "get_params", lambda: {})()
        if isinstance(params, dict):
            mlflow.log_params(params)

        mlflow.log_metric("accuracy", accuracy)

        lines = report.strip().split("\n")
        for line in lines[2:-3]: 
            parts = [x for x in line.split() if x]
            if len(parts) >= 4:
                label, precision, recall, f1, support = parts[0:5]
                try:
                    mlflow.log_metric(f"{label}_precision", float(precision))
                    mlflow.log_metric(f"{label}_recall", float(recall))
                    mlflow.log_metric(f"{label}_f1", float(f1))
                except:
                    pass

        import joblib
        joblib.dump(model_obj, f"{model_name}_model.pkl")
        mlflow.log_artifact(f"{model_name}_model.pkl")

        # Log small sample of training data
        sample = pd.DataFrame(X_train[:5])
        sample['target'] = y_train[:5]
        sample.to_csv(f"{model_name}_train_sample.csv", index=False)
        mlflow.log_artifact(f"{model_name}_train_sample.csv")

        print(f"Logged {model_name} to MLflow (DagsHub).")


In [21]:
# with mlflow.start_run(run_name="dagshub_test_run"):
#     mlflow.log_param("test_param", 1)
#     mlflow.log_metric("test_metric", 0.99)

# print("✅ DagsHub connection successful!")


In [22]:
# import tempfile

# with tempfile.NamedTemporaryFile("w", delete=False, suffix=".txt") as tmp:
#     tmp.write("This is a test artifact.")
#     temp_file_path = tmp.name

# with mlflow.start_run(run_name="dagshub_test_run"):
#     mlflow.log_param("test_param", 1)
#     mlflow.log_metric("test_metric", 0.99)
#     mlflow.log_artifact(temp_file_path)

# print("✅ DagsHub connection + artifact upload successful!")


In [23]:
results = {}

for name, model in models.items():
    try:
        print(f"\n{'='*80}\nTraining {name}...\n{'='*80}")
        t0 = time()
        model.fit(X_train_arr, y_train_arr)
        duration = time() - t0
        preds = model.predict(X_test_arr)
        
        acc = accuracy_score(y_test_arr, preds)
        report = classification_report(y_test_arr, preds, target_names=target_classes, zero_division=0)
        
        print(f"Accuracy: {acc:.4f}")
        print(report)
        
        results[name] = {"model": model, "accuracy": acc, "report": report, "train_time": duration}
        
        log_to_mlflow(name, model, acc, report, X_train_arr, y_train_arr)
        
    except Exception as e:
        print(f"Error training {name}: {e}")



Training Gaussian Naive Bayes...
Accuracy: 0.6321
              precision    recall  f1-score   support

     100-150       0.53      0.35      0.42      1948
     150-200       0.59      0.58      0.58      2199
     200-250       0.84      0.81      0.82      2428
      50-100       0.48      0.89      0.63       914

    accuracy                           0.63      7489
   macro avg       0.61      0.66      0.61      7489
weighted avg       0.64      0.63      0.62      7489

Logged Gaussian Naive Bayes to MLflow (DagsHub).
🏃 View run Gaussian Naive Bayes at: http://127.0.0.1:5000/#/experiments/977387589067566005/runs/9ddd6729691c4c2699d2c49efb7446db
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/977387589067566005

Training Logistic Regression...
Accuracy: 0.7642
              precision    recall  f1-score   support

     100-150       0.71      0.72      0.71      1948
     150-200       0.70      0.70      0.70      2199
     200-250       0.87      0.87      0.87   

C:\Users\KHUSH\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:199: UserWarning: [16:20:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.8495
              precision    recall  f1-score   support

     100-150       0.82      0.81      0.82      1948
     150-200       0.80      0.81      0.81      2199
     200-250       0.91      0.91      0.91      2428
      50-100       0.85      0.86      0.86       914

    accuracy                           0.85      7489
   macro avg       0.85      0.85      0.85      7489
weighted avg       0.85      0.85      0.85      7489

Logged XGBoost to MLflow (DagsHub).
🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/977387589067566005/runs/a2833b2bca344a8a8c4e90ffca61b8f6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/977387589067566005

Training LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000975 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 60
[LightGBM] [Info] Number of data points

C:\Users\KHUSH\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.8502
              precision    recall  f1-score   support

     100-150       0.82      0.81      0.82      1948
     150-200       0.80      0.81      0.81      2199
     200-250       0.92      0.91      0.91      2428
      50-100       0.84      0.87      0.86       914

    accuracy                           0.85      7489
   macro avg       0.85      0.85      0.85      7489
weighted avg       0.85      0.85      0.85      7489

Logged LightGBM to MLflow (DagsHub).
🏃 View run LightGBM at: http://127.0.0.1:5000/#/experiments/977387589067566005/runs/123ff85e17414905bc87577212b8d7fa
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/977387589067566005


In [24]:
summary = sorted([(n, r["accuracy"]) for n, r in results.items()], key=lambda x: x[1], reverse=True)
print("\n\nModel Accuracy Summary:")
for n, a in summary:
    print(f"{n:25s} -> {a:.4f}")

print("\nAll runs are logged to MLflow experiment:", EXPERIMENT_NAME)




Model Accuracy Summary:
LightGBM                  -> 0.8502
XGBoost                   -> 0.8495
Support Vector Machine    -> 0.8355
Random Forest             -> 0.8293
Logistic Regression       -> 0.7642
Gaussian Naive Bayes      -> 0.6321

All runs are logged to MLflow experiment: Beverage Price Prediction


In [25]:
# Find the model with the highest accuracy
best_model_name = max(results, key=lambda x: results[x]["accuracy"])
best_model_info = results[best_model_name]
best_model = best_model_info["model"]

print(f"\nBest Model: {best_model_name}")
print(f"Accuracy: {best_model_info['accuracy']:.4f}")



Best Model: LightGBM
Accuracy: 0.8502


In [26]:
import joblib
joblib.dump(best_model, f"best_model_{best_model_name}.pkl")
print(f"💾 Best model saved as: best_model_{best_model_name}.pkl")

💾 Best model saved as: best_model_LightGBM.pkl


In [27]:
# Save the list of final features (columns in X_train_final)
trained_features = list(X_train_final.columns)
joblib.dump(trained_features, "trained_feature_columns.pkl")



['label_encoder_target.pkl']

In [28]:
# Save the scaler
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

In [29]:

# Save target label encoder
joblib.dump(le_target, "label_encoder_target.pkl")

['label_encoder_target.pkl']